In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pandas as pd
import numpy as np
import csv

spark = SparkSession.builder.appName("project").getOrCreate()

In [2]:
'''
import  pyspark.sql.functions as psf
from pyspark.sql.types import DoubleType
cr_udf = psf.udf(lambda x,y:float((x-y) / x), DoubleType())
'''

'\nimport  pyspark.sql.functions as psf\nfrom pyspark.sql.types import DoubleType\ncr_udf = psf.udf(lambda x,y:float((x-y) / x), DoubleType())\n'

In [3]:
with open("/home/hz2558/Median.csv") as csvfile:
    csvReader = csv.reader(csvfile)
    Data = list(csvReader)
title = Data[0]
Data = Data[1:]
for i in range(len(Data)):
    Data[i] = list((map(eval, Data[i])))
data = np.array(Data)
zipcode = data[:, 0]
data = np.delete(data, 0, axis = 1)

In [4]:
data

array([[ 199,  199,  200, ..., 1345, 1340, 1338],
       [ 155,  156,  156, ...,  477,  477,  479],
       [ 353,  355,  356, ..., 1639, 1622, 1612],
       ...,
       [  38,   39,   39, ...,   85,   85,   85],
       [ 124,  124,  125, ...,  368,  374,  380],
       [  73,   73,   73, ...,  151,  153,  155]])

In [5]:
interval = 6
i = 0
res = np.array([[0.0]] * len(data))
while i < 270 - interval:
    res = np.insert(res, len(res[0]), (data[:,i+interval] - data[:,i]) / data[:,i], axis = 1)
    i = i + 1
res = np.delete(res, 0, axis = 1) * 100

In [6]:
D = pd.DataFrame(res)
D

,0,1,2,3,4,5,6,7,8,9,...,254,255,256,257,258,259,260,261,262,263
0,0.502513,1.005025,1.000000,1.500000,2.512563,2.500000,3.000000,2.487562,2.475248,1.970443,...,0.833965,0.453172,0.527506,0.225056,0.074627,0.896861,1.578947,1.127820,0.449775,0.149701
1,1.935484,1.282051,1.923077,1.273885,1.910828,1.265823,1.898734,1.898734,1.886792,1.886792,...,0.213220,0.854701,1.713062,2.141328,1.923077,1.489362,1.489362,1.059322,0.421053,0.419287
2,1.983003,1.690141,1.966292,2.240896,2.513966,2.785515,3.055556,3.324100,3.581267,3.835616,...,-2.998847,-3.705848,-3.902155,-3.747073,-3.225806,-2.237927,-1.486326,-1.443175,-1.696970,-1.946472
3,-1.492537,-4.411765,-4.411765,-4.477612,-4.477612,-5.970149,-4.545455,-3.076923,-1.538462,1.562500,...,0.909091,0.909091,0.909091,1.818182,1.818182,1.818182,0.900901,1.801802,1.801802,0.892857
4,1.522843,1.010101,0.000000,0.000000,-0.500000,-0.500000,-1.000000,-1.500000,-2.000000,-1.500000,...,0.984252,1.176471,1.960784,2.352941,2.152642,1.949318,1.949318,1.550388,0.961538,0.766284
5,1.851852,1.851852,0.000000,0.000000,1.851852,1.851852,0.000000,0.000000,1.818182,1.851852,...,1.176471,1.176471,2.352941,1.162791,1.162791,1.162791,2.325581,2.325581,2.298851,2.298851
6,2.040816,2.040816,2.040816,2.040816,2.040816,-2.000000,-2.000000,-2.000000,0.000000,0.000000,...,3.488372,3.488372,4.651163,4.597701,4.545455,3.409091,2.247191,3.370787,2.222222,1.098901
7,2.040816,0.000000,2.040816,0.000000,0.000000,-4.000000,-4.000000,-4.000000,-4.000000,-2.040816,...,1.204819,1.204819,2.409639,2.409639,1.190476,2.380952,2.380952,2.380952,2.352941,3.529412
8,0.751880,0.374532,1.123596,1.872659,2.247191,2.621723,2.611940,2.985075,2.222222,1.470588,...,-0.140056,-0.766017,-0.556715,-0.691563,-0.276434,0.486449,1.051893,0.701754,0.349895,-0.069638
9,-1.149425,-1.136364,-1.136364,-1.136364,0.000000,0.000000,1.162791,1.149425,1.149425,1.149425,...,3.252033,2.811245,3.600000,3.984064,3.162055,2.755906,3.543307,3.125000,2.702703,2.681992


In [7]:
D.to_csv("test_median.csv", index = False, sep=',')

df = spark.read.csv("/home/hz2558/test_median.csv", inferSchema=True, encoding='utf-8')

In [8]:
df.show(3)

+------------------+------------------+------------------+------------------+-----------------+------------------+----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------

In [9]:
from pyspark.ml.feature import VectorAssembler
vecAss = VectorAssembler(inputCols=df.columns[:-1], outputCol='features')
df_va = vecAss.transform(df)
training = df_va.select(df_va._c263, df_va.features)
          .selectExpr("_c263 as label", "features as features")
training.show(5) 

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|              263.0|[0.0,1.0,2.0,3.0,...|
|0.14970059880239522|[0.50251256281407...|
|0.41928721174004197|[1.93548387096774...|
|-1.9464720194647203|[1.98300283286118...|
| 0.8928571428571428|[-1.4925373134328...|
+-------------------+--------------------+
only showing top 5 rows



In [10]:
from pyspark.sql import SQLContext
s = training.toPandas()
s = s[1:]
training = spark.createDataFrame(s)
training.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|0.14970059880239522|[0.50251256281407...|
|0.41928721174004197|[1.93548387096774...|
|-1.9464720194647203|[1.98300283286118...|
| 0.8928571428571428|[-1.4925373134328...|
| 0.7662835249042145|[1.52284263959390...|
+-------------------+--------------------+
only showing top 5 rows



In [558]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)
'''
# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
'''

'\n# Print the coefficients and intercept for linear regression\nprint("Coefficients: %s" % str(lrModel.coefficients))\nprint("Intercept: %s" % str(lrModel.intercept))\n\n# Summarize the model over the training set and print out some metrics\ntrainingSummary = lrModel.summary\nprint("numIterations: %d" % trainingSummary.totalIterations)\nprint("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))\ntrainingSummary.residuals.show()\nprint("RMSE: %f" % trainingSummary.rootMeanSquaredError)\nprint("r2: %f" % trainingSummary.r2)\n'

In [559]:
result1 = lrModel.transform(training)
result1.show(5)

+------------------+--------------------+-----------------+
|             label|            features|       prediction|
+------------------+--------------------+-----------------+
| 5.959701068962255|[15.2685895739613...|7.311517417554945|
| 2.698558724317694|[1.53272576636288...|3.889370204902093|
|6.9436539556061465|[11.1802378774016...|8.252068940300452|
| 9.936305732484078|[-0.29296875,-0.1...|9.085775794805514|
|  7.43801652892562|[4.45859872611465...| 7.87077347415054|
+------------------+--------------------+-----------------+
only showing top 5 rows



In [560]:
trainingSummary1 = lrModel.summary
print("RMSE: %f" % trainingSummary1.rootMeanSquaredError)

RMSE: 1.830069


In [11]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)
glrModel = glr.fit(training)
result2 = glrModel.transform(training)
result2.show(5)

# Print the coefficients and intercept for generalized linear regression model
print("Coefficients: " + str(glrModel.coefficients))
print("Intercept: " + str(glrModel.intercept))

summary = glrModel.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()


Coefficients: [-0.0003329166872199775,0.0030802125133975447,-0.005055967144483851,-0.007458342721170586,-0.00048662208636622854,0.004060036817266207,-0.0018043987419907545,0.020621029443994594,0.0017438259436839813,-0.003333699291853531,-0.0032356492849853233,0.0015520864103432815,0.002131741936172896,-0.0036243468245758153,-0.0016746724320309812,0.003736409529823853,-0.0012463620135361735,-0.005149631988321004,0.00526864557289345,-0.0034240577833653946,0.005600056947864531,-0.0009379678315368944,0.00282190142266505,-8.905664996721034e-05,-0.0054152684564428896,-0.005463374173463312,0.0005740776599605427,-0.0005493412126390925,0.006039038826387222,-0.001769879683896303,0.003480696967796244,0.009148051111518136,-0.006319459518928621,-0.006174501862177245,-0.0035771538781977047,-2.5865189445238045e-05,0.001823103255131523,0.002420804111204835,0.0058873775328560805,-0.0032436873482853457,-0.00368316229213357,-0.003855995427785846,-0.0070082248191980084,-0.0052796923080710816,0.00471849646

P Values: [0.9451009406383628, 0.5639177605479557, 0.3517702911359919, 0.16564246590051512, 0.9287559408703305, 0.45798158259496247, 0.7458073287254812, 0.0002658846474921095, 0.753883590529564, 0.5388713422729625, 0.5534976601719415, 0.7776367340425239, 0.6983448233516982, 0.513744575224595, 0.7580537526463782, 0.4849839269161349, 0.8165106652939649, 0.3362035272732329, 0.32503298374724343, 0.525137151235642, 0.3004648739855167, 0.8637821525795575, 0.613305743604805, 0.9875426129702483, 0.3548985091334944, 0.35677677357940496, 0.9238693032623013, 0.9272806425917941, 0.3212691798849665, 0.7764342989375601, 0.576727946750593, 0.14791797830222553, 0.31271807764691495, 0.32233526232717535, 0.5737363589685649, 0.9967172034309428, 0.7716784705411905, 0.6979307985825614, 0.33502424699137, 0.5869262085358684, 0.5355707443050701, 0.5164251926300478, 0.23711111446249977, 0.3720695738155426, 0.4330290170148188, 0.11953312456594123, 0.8510673833878191, 0.8617019680183506, 0.8516321144235905, 0.38

In [12]:
result2 = glrModel.transform(training)
result2.show(5)

+-------------------+--------------------+-------------------+
|              label|            features|         prediction|
+-------------------+--------------------+-------------------+
|0.14970059880239522|[0.50251256281407...| 0.3740060777528322|
|0.41928721174004197|[1.93548387096774...|0.25990039028851286|
|-1.9464720194647203|[1.98300283286118...|-1.2039432702587802|
| 0.8928571428571428|[-1.4925373134328...| 1.7536532202572543|
| 0.7662835249042145|[1.52284263959390...| 0.5601081357024311|
+-------------------+--------------------+-------------------+
only showing top 5 rows



In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
glr_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = glr_evaluator.evaluate(result2)
print("RMSE: %f" % rmse)

RMSE: 0.989413


In [564]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(maxDepth=5, varianceCol="variance")
dt_model = dt.fit(training)

In [565]:
result3 = dt_model.transform(training)
result3.show(5)

+------------------+--------------------+-----------------+------------------+
|             label|            features|       prediction|          variance|
+------------------+--------------------+-----------------+------------------+
| 5.959701068962255|[15.2685895739613...|6.369308856492961|0.9352157356596654|
| 2.698558724317694|[1.53272576636288...|2.261948541746845|1.1339480003268776|
|6.9436539556061465|[11.1802378774016...|7.708100851034069|0.8538218701245323|
| 9.936305732484078|[-0.29296875,-0.1...| 8.84517894310768|0.8114052394983328|
|  7.43801652892562|[4.45859872611465...|6.369308856492961|0.9352157356596654|
+------------------+--------------------+-----------------+------------------+
only showing top 5 rows



In [566]:
from pyspark.ml.evaluation import RegressionEvaluator
dt_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = dt_evaluator.evaluate(result3)
print("RMSE: %f" % rmse)

RMSE: 1.602008


In [567]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(maxIter=10, maxDepth=3)
gbt_model = gbt.fit(training)

In [568]:
result4 = gbt_model.transform(training)
result4.show(5)

+------------------+--------------------+------------------+
|             label|            features|        prediction|
+------------------+--------------------+------------------+
| 5.959701068962255|[15.2685895739613...| 6.103809484905478|
| 2.698558724317694|[1.53272576636288...|1.2728534867693335|
|6.9436539556061465|[11.1802378774016...| 6.425519984454572|
| 9.936305732484078|[-0.29296875,-0.1...|10.072524584041583|
|  7.43801652892562|[4.45859872611465...| 6.252680751843799|
+------------------+--------------------+------------------+
only showing top 5 rows



In [569]:
from pyspark.ml.evaluation import RegressionEvaluator
gbt_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = gbt_evaluator.evaluate(result4)
print("RMSE: %f" % rmse)

RMSE: 1.850431


In [570]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(numTrees=10, maxDepth=5, seed=101)
rf_model = rf.fit(training)

In [571]:
result5 = rf_model.transform(training)
result5.show(5)

+------------------+--------------------+-----------------+
|             label|            features|       prediction|
+------------------+--------------------+-----------------+
| 5.959701068962255|[15.2685895739613...|6.555437039654312|
| 2.698558724317694|[1.53272576636288...| 2.96618654057995|
|6.9436539556061465|[11.1802378774016...|7.033299844152678|
| 9.936305732484078|[-0.29296875,-0.1...| 9.01131627774126|
|  7.43801652892562|[4.45859872611465...|6.739390097513466|
+------------------+--------------------+-----------------+
only showing top 5 rows



In [572]:
from pyspark.ml.evaluation import RegressionEvaluator
rf_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = rf_evaluator.evaluate(result5)
print("RMSE: %f" % rmse)

RMSE: 1.832458


In [573]:
s = result2.toPandas()

In [574]:
D["prediction"] = s['prediction']
D

,0,1,2,3,4,5,6,7,8,9,...,237,238,239,240,241,242,243,244,245,prediction
0,15.268590,16.732647,18.361433,19.984223,21.659228,23.367427,25.072046,26.713762,28.314196,29.789446,...,8.251147,7.786925,6.997981,7.038881,7.411303,7.028266,6.743609,6.336088,5.959701,6.477610
1,1.532726,3.180662,5.345502,8.192342,11.890661,16.310409,21.132075,25.940029,30.272597,33.940556,...,-1.352182,-0.092308,0.952966,1.132537,1.189024,1.462077,1.953602,2.355460,2.698559,3.026581
2,11.180238,11.978122,12.847538,13.845596,14.859002,15.966083,17.104789,18.320748,19.508696,20.668096,...,6.743362,7.351351,7.931950,7.793044,7.281034,7.305577,7.544876,7.239754,6.943654,7.587644
3,-0.292969,-0.195503,-0.195695,0.000000,0.196078,0.392542,0.490677,0.686948,0.981354,1.373896,...,8.903226,9.090909,8.798972,8.317338,8.365262,8.402292,8.460560,9.241555,9.936306,9.467180
4,4.458599,4.731574,4.995459,5.344203,5.781391,6.486486,7.271095,8.325873,9.553571,11.051693,...,5.677347,5.882353,6.326988,6.595864,6.451613,6.430155,6.806862,7.115279,7.438017,7.224263
5,-0.082645,0.247934,0.578035,0.989283,1.481481,2.134647,3.035275,4.016393,5.245902,6.644791,...,4.885993,5.297297,5.768194,6.011809,6.026667,5.944798,6.250000,6.832627,7.089947,7.043883
6,24.380496,24.693312,25.029377,25.494761,26.000000,26.592903,27.289932,27.938415,28.645833,29.323032,...,17.976719,15.530303,13.367035,13.310459,13.516718,10.742348,7.565067,6.198536,4.933544,6.302822
7,30.973699,32.553172,34.091540,35.572322,36.988558,38.312398,39.511533,40.627247,41.619888,42.467807,...,-1.818457,-0.588772,3.936742,8.073957,9.528968,10.134203,10.986091,12.073887,12.619145,12.437411
8,7.513298,7.427056,7.407407,7.321900,7.171053,7.086614,6.932636,6.779661,6.623377,6.468305,...,23.347398,23.953488,23.735051,23.800822,23.839854,23.662738,23.388914,23.600537,23.930481,23.080594
9,1.355014,1.352814,1.350621,1.403130,1.454741,1.560818,1.666667,1.772288,1.932367,2.145923,...,7.210145,6.733885,5.821429,4.487633,3.753069,4.124432,4.596100,4.498270,4.452882,4.731882


In [575]:
N = len(Data)
col = pd.DataFrame(["CR"] * N)
D.insert(0,'CR',col)

In [576]:
columns = ["Date"] + title[1:-24] + ["predictions"]
D.columns = columns

In [577]:
D

,Date,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,predictions
0,CR,15.268590,16.732647,18.361433,19.984223,21.659228,23.367427,25.072046,26.713762,28.314196,...,8.251147,7.786925,6.997981,7.038881,7.411303,7.028266,6.743609,6.336088,5.959701,6.477610
1,CR,1.532726,3.180662,5.345502,8.192342,11.890661,16.310409,21.132075,25.940029,30.272597,...,-1.352182,-0.092308,0.952966,1.132537,1.189024,1.462077,1.953602,2.355460,2.698559,3.026581
2,CR,11.180238,11.978122,12.847538,13.845596,14.859002,15.966083,17.104789,18.320748,19.508696,...,6.743362,7.351351,7.931950,7.793044,7.281034,7.305577,7.544876,7.239754,6.943654,7.587644
3,CR,-0.292969,-0.195503,-0.195695,0.000000,0.196078,0.392542,0.490677,0.686948,0.981354,...,8.903226,9.090909,8.798972,8.317338,8.365262,8.402292,8.460560,9.241555,9.936306,9.467180
4,CR,4.458599,4.731574,4.995459,5.344203,5.781391,6.486486,7.271095,8.325873,9.553571,...,5.677347,5.882353,6.326988,6.595864,6.451613,6.430155,6.806862,7.115279,7.438017,7.224263
5,CR,-0.082645,0.247934,0.578035,0.989283,1.481481,2.134647,3.035275,4.016393,5.245902,...,4.885993,5.297297,5.768194,6.011809,6.026667,5.944798,6.250000,6.832627,7.089947,7.043883
6,CR,24.380496,24.693312,25.029377,25.494761,26.000000,26.592903,27.289932,27.938415,28.645833,...,17.976719,15.530303,13.367035,13.310459,13.516718,10.742348,7.565067,6.198536,4.933544,6.302822
7,CR,30.973699,32.553172,34.091540,35.572322,36.988558,38.312398,39.511533,40.627247,41.619888,...,-1.818457,-0.588772,3.936742,8.073957,9.528968,10.134203,10.986091,12.073887,12.619145,12.437411
8,CR,7.513298,7.427056,7.407407,7.321900,7.171053,7.086614,6.932636,6.779661,6.623377,...,23.347398,23.953488,23.735051,23.800822,23.839854,23.662738,23.388914,23.600537,23.930481,23.080594
9,CR,1.355014,1.352814,1.350621,1.403130,1.454741,1.560818,1.666667,1.772288,1.932367,...,7.210145,6.733885,5.821429,4.487633,3.753069,4.124432,4.596100,4.498270,4.452882,4.731882


In [578]:
D.loc[0, :]

Date                CR
1996-04        15.2686
1996-05        16.7326
1996-06        18.3614
1996-07        19.9842
1996-08        21.6592
1996-09        23.3674
1996-10         25.072
1996-11        26.7138
1996-12        28.3142
1997-01        29.7894
1997-02        31.1998
1997-03        32.5491
1997-04        33.7974
1997-05        34.8742
1997-06        35.8987
1997-07        36.7845
1997-08        37.5126
1997-09        38.1179
1997-10        38.5725
1997-11        38.9313
1997-12        39.1294
1998-01        39.1992
1998-02        39.1777
1998-03         39.096
1998-04        38.9118
1998-05        38.6163
1998-06         38.215
1998-07        37.7822
1998-08        37.3112
                ...   
2014-05        7.83748
2014-06        7.50436
2014-07        7.41803
2014-08        7.38549
2014-09        7.48348
2014-10        7.87042
2014-11        8.31308
2014-12        8.54287
2015-01        8.20632
2015-02        8.18778
2015-03        8.83566
2015-04        8.97346
2015-05    

In [579]:
#a.to_csv("./Median_6month/a.csv", index=True, sep=",")
for i in range(N):
    name = str(zipcode[i])
    if len(name) == 4:
        name = '0' + name
    one_data = D.loc[i, :]
    one_data.to_csv("./nb3/4B_2year/{0}.csv".format(name), index=True, sep=",")
    